# Introduction : Graph Based Trajectory Prediction

이 프로젝트는 GNN 기반 모델을 사용하여, 경계-보안 지역을 대상으로 사람의 미래 위치를 예측하는 프로젝트입니다.

관련한 논문은 [여기](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE12050314) 에서 확인할 수 있습니다.

## 1. Data Collection

프로젝트를 시작하기에 앞서, 시나리오에 맞는 데이터들을 수집해야 했습니다. 시나리오는 아래와 같습니다.

- 해안, 비무장지대 등 군사지역에 준하는 배경
- 사람이 등장하여 장애물(돌, 나무 등)뒤에 숨거나 가려짐
- 관찰자 시점(CCTV)에서 촬영하며, 화면은 고정된 상태일 것
- 비디오의 길이는 무관, 프레임은 10FPS, 해상도는 1920x1080

따라서 이 조건에 맞는 데이터를 구축하기 위해 GTA5 게임을 사용하여 상황을 시뮬레이션 하였습니다.

아래는 생성한 데이터의 예시 이미지 입니다.

![예제 이미지1](./source/examples/ex2.png)

생성한 원본 데이터는 ./source/original_vids 폴더에서 확인할 수 있습니다.

각 비디오의 속성은 아래와 같습니다.

- case 1 : 낮, 해안가, 돌, 사람, 1920X1080, 30FPS
- case 2 : 밤, 해안가, 돌, 사람, 1920X1080, 30FPS
- case 3 : 낮, 도로, 펜스, 사람, 1920X1080, 30FPS
- case 4 : 낮, 도로, 자동차, 사람, 1920X1080, 30FPS

### 1.1 Video Compression

기본적으로 30FPS으로 촬영된 원본 비디오를 그대로 사용하는건 비효율적입니다. 따라서 비디오의 프레임과 화질을 낮춰 압축하였습니다.

> 이 챕터의 작업을 수행하기 위해 다음과 같은 프로그램이 필요합니다. 
> 
> ※주의 - 이 워크스루는 Ubuntu 22.04 LTS 환경을 기준으로 작성되었습니다. 다른 OS를 사용할 경우 환경 변수 설정 등 추가적인 설치가 필요할 수 있습니다.
> 1. [ffmpeg](https://www.ffmpeg.org/) - windows/mac/linux
> 2. [anaconda3](https://www.anaconda.com/download) - windows/mac/linux

가장 먼저 비디오를 압축하기 위해 ffmpeg를 설치합니다.
```bash
sudo apt-get update
sudo apt install ffmpeg -y
```

비디오를 다음 명령어를 사용하여 압축합니다.
```bash
ffmpeg -i source/original_vids/case_1.mp4 -an -r 10 copy output/case_1_comp.mp4
```
- i source/original_vids/case_1.mp4 → 입력 파일 지정
- an → 오디오 제거
- r 10 → 프레임 속도를 10FPS로 변경
- output/case_1_comp.mp4 → 출력 파일 지정

다음으로, 비디오의 프레임을 한 장씩 개별 이미지로 저장합니다.
```bash
mkdir -p output/images
ffmpeg -i output/case_1_comp.mp4 output/images/case_1_%04d.png
```

### 1.2 Data Annotation

데이터 어노테이션이란 데이터셋에 [메타데이터](https://ko.wikipedia.org/wiki/%EB%A9%94%ED%83%80%EB%8D%B0%EC%9D%B4%ED%84%B0) 를 정의하는 작업입니다.

저는 데이터 어노테이션 자동화를 위해 [YOLOv11](https://docs.ultralytics.com/ko/models/yolo11/)을 사용하였습니다.

> 이 챕터의 작업을 수행하기 위해 다음과 같은 프로그램이 필요합니다. 
>
> ※주의 - 이 워크스루는 Ubuntu 22.04 LTS 환경을 기준으로 작성되었습니다. 다른 OS를 사용할 경우 환경 변수 설정 등 추가적인 설치가 필요할 수 있습니다.
> 1. [anaconda3](https://www.anaconda.com/download) - windows/mac/linux
> 2. [ultralytics](https://github.com/ultralytics/ultralytics) - python >=3.8
> 3. [pyside6](https://pypi.org/project/PySide6/) - python >=3.9
> 4. (선택) [CUDA](https://starlane.tistory.com/1) - nvidia 그래픽카드 한정, GPU를 사용하여 YOLO를 돌릴 때 사용
> 5. [scikit-learn](https://scikit-learn.org/stable/)
> 6. [networkX](https://networkx.org/)

가장 먼저 PC에 anaconda를 설치합니다.
```bash
# 아나콘다 설치 스크립트 다운로드
wget https://repo.anaconda.com/archive/Anaconda3-latest-Linux-x86_64.sh

# 설치 스크립트 실행
bash Anaconda3-latest-Linux-x86_64.sh

# 환경 변수 적용
source ~/.bashrc

# conda 명령어가 인식되지 않을 경우, bashrc에 환경 변수 직접 추가
echo 'export PATH="$HOME/anaconda3/bin:$PATH"' >> ~/.bashrc
source ~/.bashrc

# 설치가 잘 되었는지 확인
conda --version
```

다음으로 python 3.11 버전을 기준으로 conda 가상환경을 생성합니다.
```bash
# 가상환경 생성
conda create -n graph python=3.11

# 가상환경 활성화
conda activate graph

# 환경 활성화 확인
conda info --envs

# 만약 가상환경을 종료하고 싶다면
conda deactivate
```

> ※ 선택사항 - Yolo모델은 GPU 환경에서 더욱 빠르게 동작합니다. GPU 가속을 위해 CUDA 및 Pytorch 설치가 필요합니다.
> 
> GPU가 없거나, CPU를 사용하는 방법은 1.2.1 챕터에서 설명하고 있습니다.
> 이 문서는 Nvidia RTX3060 환경을 기준으로 작성하였습니다.

GPU 가속을 위해 CUDA 설치가 필요합니다. CUDA는 Nvidia의 그래픽카드만 지원합니다. 

자신의 그래픽카드 버전을 확인하고, 호환되는 드라이버 및 CUDA를 설치해야 합니다.

호환 목록은 [여기](https://www.wikiwand.com/en/articles/CUDA#/GPUs_supported)서 찾을 수 있습니다.

CUDA 설치와 관련한 내용은 [티스토리](https://starlane.tistory.com/1) 를 참고하기시 바랍니다.

nvidia 그래픽 드라이버 및 cuda 설치가 완료되었다면, 콘다 가상환경에 pytorch를 설치해야 합니다.
```bash
conda activate graph

# CUDA 12.0 이상 버전의 경우
pip3 install torch torchvision torchaudio

# CUDA 11.8 버전의 경우
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```

Pytorch 설치가 완료되었다면, Yolo 모델을 사용하기 위해 ultralytics 패키지 설치가 필요합니다.
```bash
pip3 install ultralytics
```

### 1.2.1 Non-GPU Enviroments

> 이 챕터는 GPU 가속을 사용하지 않는 환경에 한정됩니다.

conda 환경에서 시작합니다.
```bash
# 가상환경 활성화
conda activate graph

# pytorch 설치
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# ultralytics 설치
pip3 install ultralytics
```

### 1.2.2 Install Pyside6
> Pyside6는 Ubuntu Server, CLI 환경에서 사용하기 어렵거나나 추가적인 설치가 필요할 수 있습니다.
>

```bash
# Pyside6 설치
pip3 install pyside6
```

### 1.2.3 Auto Annotation Using Yolov11

이제 모든 설치가 완료되었습니다. 이 챕터에서는 Yolov11 모델을 사용하여 자동으로 어노테이션하는 코드를 담고 있습니다.

Annotation의 목표는, 사진으로 쪼개진 비디오에서 사람을 빠르게 인식하여 Bounding Box(이하 Bbox) 정보를 담은 메타데이터를 생성하는 것입니다.

In [ ]:
# =======================
# YOLO_Detect.py
# 이 코드는 이미지 파일들에서 사람 객체를 탐지하고, 해당 위치를 KITTI 포맷으로 저장함
# =======================

import os
from ultralytics import YOLO

# YOLOv11x 모델 로드
# YOLOv11은 다양한 가중치가 있으므로, 원하는 가중치를 선택할 수 있습니다.
model = YOLO('yolo11x.pt')

# 입력 이미지 경로와 출력 라벨 경로 설정
image_folder = 'source/images/'  # 이미지가 저장된 폴더
label_folder = 'source/label/'   # 라벨 파일을 저장할 폴더
os.makedirs(label_folder, exist_ok=True)  # 출력 폴더 생성

# 클래스 ID 설정 (YOLO에서 'Person' 클래스 ID가 0번)
PERSON_CLASS_ID = 0

# 이미지 파일 리스트 가져오기
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# 라벨링 작업 시작
for image_file in image_files:
    # 이미지 경로
    image_path = os.path.join(image_folder, image_file)

    # 이미지에서 객체 탐지 실행
    results = model(image_path)

    # 라벨 데이터 파일 저장 경로
    txt_file_name = os.path.splitext(image_file)[0] + '.txt'
    txt_file_path = os.path.join(label_folder, txt_file_name)

    # 라벨 데이터 저장
    with open(txt_file_path, 'w') as f:
        # 결과 가져오기
        for result in results:
            boxes = result.boxes  # Bounding box 정보
            for box in boxes:
                # Box 정보 추출
                x_center, y_center, width, height = box.xywhn[0].tolist()  # Normalized 값
                class_id = int(box.cls[0])  # 클래스 ID

                # Person 클래스만 저장
                if class_id == PERSON_CLASS_ID:
                    # YOLO txt 형식: class_id x_center y_center width height
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # 아무것도 검출되지 않아도 빈 파일 생성
    print(f"Processed: {image_file}, Saved: {txt_file_path}")

print("모든 이미지 처리 완료.")

YOLO 모델은 꽤 높은 정확도를 보여주지만, 라벨링 된 데이터를 수동으로 확인하여 오류를 보정하고, 제거해야 합니다.

이때, 아래 코드를 사용하여 GUI 환경에서 쉽게 데이터를 편집할 수 있습니다.

"YOLO 라벨링 도구" 에 대한 설명은 아래 사진에서 확인할 수 있습니다.


![예제 이미지3](./source/examples/ex3.png)

![예제 이미지4](./source/examples/ex4.png)

In [ ]:
# =======================
# Data_Labeler.py
# 이 코드는 이미지와 라벨 파일을 읽어와 시각화하고, 데이터를 수정하는 몇 가지 기능을 제공함.
# =======================

import sys
import os
import subprocess
from PySide6.QtWidgets import (
    QApplication, QMainWindow, QLabel, QVBoxLayout, QHBoxLayout,
    QPushButton, QLineEdit, QWidget, QFileDialog, QMessageBox, QScrollArea, QSlider,
    QListWidget, QInputDialog, QDialog
)
from PySide6.QtGui import QPixmap, QPainter, QPen, QClipboard
from PySide6.QtCore import Qt, QPoint, QRect

class ImageLabel(QLabel):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setScaledContents(True)
        self.start_point = None
        self.end_point = None
        self.rectangles = []
        self.image_size = (1, 1)
        self.clipboard = QApplication.clipboard()

    def reset_labels(self):
        """라벨링 데이터 초기화"""
        self.rectangles = []
        self.start_point = None
        self.end_point = None
        self.update()

    def set_image_size(self, width, height):
        self.image_size = (width, height)

    def convert_to_image_coords(self, rect):
        label_width = self.width()
        label_height = self.height()
        img_width, img_height = self.image_size

        x_scale = img_width / label_width
        y_scale = img_height / label_height

        x_min = rect.left() * x_scale
        y_min = rect.top() * y_scale
        x_max = rect.right() * x_scale
        y_max = rect.bottom() * y_scale

        return QRect(int(x_min), int(y_min), int(x_max - x_min), int(y_max - y_min))

    def get_yolo_format(self, rect, class_id):
        """YOLO 포맷으로 좌표 변환"""
        img_width, img_height = self.image_size
        
        x_min = rect.left()
        y_min = rect.top()
        x_max = rect.right()
        y_max = rect.bottom()

        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self.start_point = event.pos()
            self.end_point = self.start_point

    def mouseMoveEvent(self, event):
        if self.start_point:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton and self.start_point:
            self.end_point = event.pos()
            rect = QRect(self.start_point, self.end_point).normalized()

            rect_in_image_coords = self.convert_to_image_coords(rect)

            class_id, ok = QInputDialog.getInt(
                self, "클래스 ID 입력", "클래스 ID를 입력하세요:", minValue=0
            )
            if ok:
                self.rectangles.append((rect_in_image_coords, class_id))
                
                # YOLO 포맷으로 변환하여 클립보드에 복사
                yolo_format = self.get_yolo_format(rect_in_image_coords, class_id)
                self.clipboard.setText(yolo_format)

            self.start_point = None
            self.end_point = None
            self.update()

    def paintEvent(self, event):
        super().paintEvent(event)
        painter = QPainter(self)
        pen = QPen(Qt.red, 2, Qt.SolidLine)
        painter.setPen(pen)

        for rect, class_id in self.rectangles:
            x_scale = self.width() / self.image_size[0]
            y_scale = self.height() / self.image_size[1]
            label_rect = QRect(
                int(rect.left() * x_scale),
                int(rect.top() * y_scale),
                int(rect.width() * x_scale),
                int(rect.height() * y_scale)
            )
            painter.drawRect(label_rect)
            painter.drawText(label_rect.topLeft(), str(class_id))

        if self.start_point and self.end_point:
            rect = QRect(self.start_point, self.end_point).normalized()
            painter.drawRect(rect)

    def append_yolo_format(self, file_path):
        """기존 라벨을 유지하면서 새로운 라벨 추가"""
        # 기존 라벨 읽기
        existing_labels = []
        if os.path.exists(file_path):
            with open(file_path, "r") as file:
                existing_labels = file.readlines()
        
        # 새로운 라벨 추가
        with open(file_path, "w") as file:
            # 기존 라벨 쓰기
            for label in existing_labels:
                file.write(label.strip() + "\n")
            
            # 새로운 라벨 추가
            for rect, class_id in self.rectangles:
                yolo_format = self.get_yolo_format(rect, class_id)
                file.write(yolo_format + "\n")


class LabelingDialog(QDialog):
    def __init__(self, image_path, label_path, parent=None):
        super().__init__(parent)
        self.setWindowTitle("이미지 라벨링")
        self.setModal(True)
        self.resize(900, 700)

        self.image_label = ImageLabel()
        self.image_label.setStyleSheet("background-color: lightgray;")
        self.image_label.setFixedSize(800, 600)

        # 이미지 로드
        pixmap = QPixmap(image_path)
        self.image_label.setPixmap(pixmap)
        self.image_label.set_image_size(pixmap.width(), pixmap.height())

        self.save_button = QPushButton("저장")
        self.save_button.clicked.connect(lambda: self.save_labels(label_path))

        layout = QVBoxLayout()
        layout.addWidget(self.image_label)
        layout.addWidget(self.save_button)

        self.setLayout(layout)

    def save_labels(self, label_path):
        self.image_label.append_yolo_format(label_path)
        QMessageBox.information(self, "알림", "라벨이 저장되었습니다.")
        self.accept()


class LabelingTool(QMainWindow):
    def __init__(self):
        super().__init__()

        # Window 설정
        self.setWindowTitle("YOLO 라벨링 도구")
        self.setGeometry(100, 100, 1000, 800)

        # 이미지 및 라벨 경로
        self.image_folder = ''
        self.label_folder = ''
        self.current_image_index = 0
        self.image_files = []
        self.deleted_labels = []

        # UI 요소 설정
        self.init_ui()

    def init_ui(self):
        # 중앙 위젯과 레이아웃 생성
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # 메인 레이아웃: 수평 레이아웃
        main_layout = QHBoxLayout()
        central_widget.setLayout(main_layout)

        # 왼쪽 레이아웃 (이미지 뷰와 설정)
        left_layout = QVBoxLayout()
        main_layout.addLayout(left_layout, stretch=3)

        # 오른쪽 레이아웃 (이미지 리스트)
        right_layout = QVBoxLayout()
        main_layout.addLayout(right_layout, stretch=1)

        # 경로 입력 필드
        path_layout = QHBoxLayout()
        left_layout.addLayout(path_layout)

        # 이미지 폴더 필드
        self.image_path_field = QLineEdit()
        self.image_path_field.setPlaceholderText("이미지 폴더 경로")
        path_layout.addWidget(self.image_path_field)

        image_browse_button = QPushButton("이미지 선택")
        image_browse_button.clicked.connect(self.browse_image_folder)
        path_layout.addWidget(image_browse_button)

        # 라벨 폴더 필드
        self.label_path_field = QLineEdit()
        self.label_path_field.setPlaceholderText("라벨 폴더 경로")
        path_layout.addWidget(self.label_path_field)

        label_browse_button = QPushButton("라벨 선택")
        label_browse_button.clicked.connect(self.browse_label_folder)
        path_layout.addWidget(label_browse_button)

        # 이미지 표시용 ScrollArea
        self.scroll_area = QScrollArea()
        self.image_label = QLabel("이미지를 불러오세요")
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("border: 1px solid black;")
        self.scroll_area.setWidget(self.image_label)
        self.scroll_area.setWidgetResizable(True)
        left_layout.addWidget(self.scroll_area)

        # 스크롤 슬라이더 추가
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(0)
        self.slider.valueChanged.connect(self.scroll_images)
        left_layout.addWidget(self.slider)

        # 오른쪽 레이아웃: 이미지 리스트와 라벨 데이터
        self.image_list_widget = QListWidget()
        self.image_list_widget.itemClicked.connect(self.image_list_clicked)
        right_layout.addWidget(QLabel("이미지 리스트"))
        right_layout.addWidget(self.image_list_widget)

        self.label_data_widget = QListWidget()
        right_layout.addWidget(QLabel("라벨 데이터"))
        right_layout.addWidget(self.label_data_widget)

        # 버튼 레이아웃
        button_layout = QHBoxLayout()
        left_layout.addLayout(button_layout)

        load_button = QPushButton("불러오기")
        load_button.clicked.connect(self.load_images)
        button_layout.addWidget(load_button)

        open_button = QPushButton("txt파일 열기")
        open_button.clicked.connect(self.open_txt)
        button_layout.addWidget(open_button)

        insert_all_button = QPushButton("txt파일 일괄 삽입")
        insert_all_button.clicked.connect(self.insert_all_txt)
        button_layout.addWidget(insert_all_button)

        labeling_button = QPushButton("현재 파일 라벨링")
        labeling_button.clicked.connect(self.open_labeling_dialog)
        button_layout.addWidget(labeling_button)

    def browse_image_folder(self):
        folder = QFileDialog.getExistingDirectory(self, "이미지 폴더 선택")
        if folder:
            self.image_path_field.setText(folder)

    def browse_label_folder(self):
        folder = QFileDialog.getExistingDirectory(self, "라벨 폴더 선택")
        if folder:
            self.label_path_field.setText(folder)

    def load_images(self):
        # 이미지 및 라벨 폴더 읽기
        self.image_folder = self.image_path_field.text()
        self.label_folder = self.label_path_field.text()

        # 이미지 파일 로드
        if not os.path.exists(self.image_folder):
            QMessageBox.warning(self, "경고", "유효한 이미지 폴더를 선택하세요!")
            return

        self.image_files = [f for f in os.listdir(self.image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.image_files.sort()

        if not self.image_files:
            QMessageBox.warning(self, "경고", "이미지 폴더에 파일이 없습니다!")
            return

        # 슬라이더 설정
        self.slider.setMaximum(len(self.image_files) - 1)
        self.slider.setValue(0)

        # 이미지 리스트 업데이트
        self.image_list_widget.clear()
        self.image_list_widget.addItems(self.image_files)

        # 첫 번째 이미지 표시
        self.current_image_index = 0
        self.show_image_with_labels()

    def show_image_with_labels(self):
        if not self.image_files:
            return

        # 현재 이미지 파일 경로
        image_file = self.image_files[self.current_image_index]
        image_path = os.path.join(self.image_folder, image_file)

        # 이미지 로드
        pixmap = QPixmap(image_path)
        if pixmap.isNull():
            QMessageBox.warning(self, "경고", f"이미지를 불러올 수 없습니다: {image_file}")
            return

        # 이미지 크기 축소 (ScrollArea의 크기에 맞춤)
        scroll_area_width = self.scroll_area.width()
        scroll_area_height = self.scroll_area.height()
        pixmap = pixmap.scaled(scroll_area_width - 20, scroll_area_height - 20, 
                             Qt.KeepAspectRatio, Qt.SmoothTransformation)

        # 라벨 파일 경로
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(self.label_folder, label_file)

        # 라벨 데이터 표시 초기화
        self.label_data_widget.clear()

        # 라벨 표시
        painter = QPainter(pixmap)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    class_id, x_center, y_center, width, height = map(float, parts)
                    # Bounding box를 픽셀 좌표로 변환
                    x_center *= pixmap.width()
                    y_center *= pixmap.height()
                    width *= pixmap.width()
                    height *= pixmap.height()
                    x = int(x_center - width / 2)
                    y = int(y_center - height / 2)

                    # 박스 그리기
                    pen = QPen(Qt.red, 2)
                    painter.setPen(pen)
                    painter.drawRect(x, y, int(width), int(height))

                    # 클래스 이름 표시
                    painter.setPen(Qt.blue)
                    painter.drawText(x, y - 5, f"Class: {int(class_id)}")

                    # 라벨 데이터를 리스트에 추가
                    self.label_data_widget.addItem(
                        f"Class: {int(class_id)}, x: {x_center:.2f}, y: {y_center:.2f}, w: {width:.2f}, h: {height:.2f}"
                    )

        painter.end()
        self.image_label.setPixmap(pixmap)

    def scroll_images(self, value):
        self.current_image_index = value
        self.show_image_with_labels()
        
        # 현재 이미지를 리스트에서 선택 상태로 만들기
        self.image_list_widget.setCurrentRow(self.current_image_index)
    
    def image_list_clicked(self, item):
        # 리스트에서 선택한 이미지의 인덱스 가져오기
        self.current_image_index = self.image_files.index(item.text())
        self.slider.setValue(self.current_image_index)
        self.show_image_with_labels()

    def open_txt(self):
        if not self.image_files:
            QMessageBox.warning(self, "경고", "먼저 이미지를 불러오세요!")
            return

        # 현재 이미지 파일 이름
        image_file = self.image_files[self.current_image_index]
        
        # 해당 이미지의 txt 파일 경로
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(self.label_folder, label_file)

        # 파일이 존재하는지 확인
        if not os.path.exists(label_path):
            QMessageBox.warning(self, "경고", "해당 이미지의 라벨 파일이 존재하지 않습니다!")
            return

        # 운영 체제에 따라 메모장 열기
        if sys.platform.startswith('win'):  # Windows
            os.startfile(label_path)
        elif sys.platform.startswith('darwin'):  # macOS
            subprocess.call(('open', label_path))
        else:  # Linux
            subprocess.call(('xdg-open', label_path))

    def insert_all_txt(self):
        # 이미지 폴더와 라벨 폴더가 설정되었는지 확인
        if not self.image_folder or not self.label_folder:
            QMessageBox.warning(self, "경고", "먼저 이미지와 라벨 폴더를 선택하세요!")
            return

        # 삽입할 텍스트 파일 선택
        insert_file, _ = QFileDialog.getOpenFileName(self, "삽입할 txt 파일 선택", "", "Text Files (*.txt)")
        
        if not insert_file:
            return

        # 삽입할 텍스트 읽기
        with open(insert_file, 'r') as f:
            insert_text = f.read().strip()

        # 모든 라벨 파일에 대해 처리
        for image_file in self.image_files:
            # 해당 이미지의 라벨 파일 경로
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(self.label_folder, label_file)

            # 기존 라벨 내용 읽기
            existing_labels = []
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    existing_labels = f.readlines()

            # 새로운 내용 추가
            with open(label_path, 'w') as f:
                # 기존 라벨들 먼저 쓰기
                f.writelines(existing_labels)
                
                # 줄바꿈 추가 (기존 내용이 있다면)
                if existing_labels and not existing_labels[-1].endswith('\n'):
                    f.write('\n')
                
                # 새로운 라벨 내용 추가
                f.write(insert_text + '\n')

        # 작업 완료 메시지
        QMessageBox.information(self, "완료", "모든 라벨 파일에 내용을 추가했습니다.")

    def open_labeling_dialog(self):
        if not self.image_files or not self.label_folder:
            QMessageBox.warning(self, "경고", "이미지와 라벨 폴더를 먼저 선택하세요!")
            return

        # 현재 이미지 파일 경로
        image_file = self.image_files[self.current_image_index]
        image_path = os.path.join(self.image_folder, image_file)

        # 라벨 파일 경로
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(self.label_folder, label_file)

        # 라벨링 다이얼로그 열기
        dialog = LabelingDialog(image_path, label_path, self)
        if dialog.exec_() == QDialog.Accepted:
            # 라벨링 후 이미지 업데이트
            self.show_image_with_labels()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    tool = LabelingTool()
    tool.show()
    sys.exit(app.exec())

### 1.3 Convert to Graph 
이제 데이터를 그래프 형태로 변환해야 합니다. 기본적으로 그래프는 JSON 형식으로 잘 표현할 수 있습니다.

이 프로젝트에서 사용한 그래프는 FC 형태로, 모든 노드가 서로 이어져 있습니다.
JSON 파일의 구조는 아래와 같습니다.

1. 기본 정보
- Image: case1_0144.png
  - 해당 프레임의 이미지 파일 이름입니다.
- label: case1_0144.txt
  - 이 프레임에 대한 레이블 정보가 저장된 파일 이름입니다.
- frame_no: 0003
  - 프레임 번호입니다.  

2. 노드(Nodes)
- 노드는 장면 내의 객체들을 나타냅니다. 각 노드는 다음과 같은 속성을 가지고 있습니다:

  - id: 객체의 고유 식별자 (예: "0", "1").
  - class: 객체의 클래스 (예: "person", "rock", "tree").
  - state: 객체의 상태 ("dynamic" 또는 "static").
    + dynamic: 움직이는 객체 (예: 사람).
    + static: 정적인 객체 (예: 바위, 나무).
  - x: 객체의 x 좌표.
  - y: 객체의 y 좌표.
  - speed: 객체의 속도 (정적인 객체는 0).
  - heading: 객체의 이동 방향 (정적인 객체는 0).
  - visual_features: 객체의 시각적 특징을 나타내는 속성들:
    + contrast: 대비.
    + dissimilarity: 비유사성.
    + homogeneity: 동질성.
    + energy: 에너지.
    + correlation: 상관관계.

3. 엣지(Edges)
- 엣지는 노드들 간의 관계를 나타냅니다. 각 엣지는 다음과 같은 속성을 가지고 있습니다:
  - id: 엣지의 고유 식별자 (예: "e1", "e2").
  - source: 엣지의 출발 노드 ID.
  - target: 엣지의 도착 노드 ID.
  - position: 두 노드 간의 상대적 위치 관계 (예: "front", "back", "left", "right", "none").

./source/examples 폴더의 ex5.json이 예시 파일입니다.

In [ ]:

# =======================
# Graph_generator.py
# 이 코드는 이미지와 라벨을 입력으로 받아, 그래프 구조를 생성하여 json으로 저장함.
# =======================

import os
import json
import math
import numpy as np
from glob import glob
from enum import Enum
from typing import Dict, Optional, Tuple
from PIL import Image
from skimage.feature import graycomatrix, graycoprops  # greycomatrix에서 graycomatrix로 변경
import cv2

class SpatialRelation(Enum):
    FRONT = 0
    BACK = 1
    LEFT = 2
    RIGHT = 3
    FRONT_LEFT = 4
    FRONT_RIGHT = 5
    BACK_LEFT = 6
    BACK_RIGHT = 7
    NONE = 8

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def determine_spatial_relation(rel_x, rel_y, distance, distance_threshold=30, angle_threshold=45):
    """
    두 객체 간의 상대적 위치를 기반으로 공간적 관계를 결정
    Missing person의 경우 항상 BACK 관계 반환
    """
    if distance > distance_threshold:
        return SpatialRelation.NONE

    angle = math.degrees(math.atan2(rel_y, rel_x))
    angle = (angle + 360) % 360

    if (45 - angle_threshold) <= angle < (45 + angle_threshold):
        return SpatialRelation.FRONT
    elif (225 - angle_threshold) <= angle < (225 + angle_threshold):
        return SpatialRelation.BACK
    elif (315 - angle_threshold) <= angle < (315 + angle_threshold):
        return SpatialRelation.RIGHT
    elif (135 - angle_threshold) <= angle < (135 + angle_threshold):
        return SpatialRelation.LEFT
    elif (315 + angle_threshold) <= angle or angle < (45 - angle_threshold):
        return SpatialRelation.FRONT_RIGHT
    elif (45 + angle_threshold) <= angle < (135 - angle_threshold):
        return SpatialRelation.FRONT_LEFT
    elif (135 + angle_threshold) <= angle < (225 - angle_threshold):
        return SpatialRelation.BACK_LEFT
    elif (225 + angle_threshold) <= angle < (315 - angle_threshold):
        return SpatialRelation.BACK_RIGHT
    else:
        return SpatialRelation.NONE


class PersonTracker:
    def __init__(self):
        self.prev_nearest_obj: Optional[Dict] = None
        self.prev_nearest_dist: float = float('inf')
        self.prev_positions = {}  # 이전 프레임의 위치 저장
        self.prev_time = None
        self.frame_rate = 30.0  # 초당 프레임 수 (필요에 따라 조정)
    
    def calculate_motion(self, current_pos, node_id, current_time):
        """
        객체의 속도와 방향을 계산합니다.
        
        Args:
            current_pos: (x, y) 현재 위치
            node_id: 객체 ID
            current_time: 현재 프레임 번호
            
        Returns:
            speed: 속도 (pixels/second)
            heading: 이동 방향 (도(degree) 단위, 0-360)
        """
        if node_id not in self.prev_positions or self.prev_time is None:
            self.prev_positions[node_id] = current_pos
            self.prev_time = current_time
            return 0, 0
        
        prev_pos = self.prev_positions[node_id]
        
        # 위치 변화 계산
        dx = current_pos[0] - prev_pos[0]
        dy = current_pos[1] - prev_pos[1]
        
        # 시간 간격 계산 (초 단위)
        dt = (current_time - self.prev_time) / self.frame_rate
        if dt == 0:
            return 0, 0
            
        # 속도 계산 (pixels/second)
        distance = math.sqrt(dx**2 + dy**2)
        speed = distance / dt if dt > 0 else 0
        
        # 방향 계산 (라디안 -> 도)
        heading = math.degrees(math.atan2(dy, dx))
        heading = (heading + 360) % 360  # 0-360도 범위로 변환
        
        # 현재 위치를 다음 계산을 위해 저장
        self.prev_positions[node_id] = current_pos
        self.prev_time = current_time
        
        return speed, heading
    
    def update(self, nodes: list, has_person: bool, current_time: int):
        """
        객체 추적 정보를 업데이트합니다.
        
        Args:
            nodes: 현재 프레임의 모든 노드 리스트
            has_person: person 객체 존재 여부
            current_time: 현재 프레임 번호
        """
        if has_person:
            person_node = next((node for node in nodes if node["class"] == "person"), None)
            if person_node:
                min_dist = float('inf')
                nearest_obj = None
                
                for node in nodes:
                    if node["class"] != "person":
                        dist = calculate_distance(
                            person_node["x"], person_node["y"],
                            node["x"], node["y"]
                        )
                        if dist < min_dist:
                            min_dist = dist
                            nearest_obj = node.copy()
                
                self.prev_nearest_obj = nearest_obj
                self.prev_nearest_dist = min_dist
        
        return self.prev_nearest_obj

def extract_glcm_features(image: np.ndarray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """
    이미지 패치에서 GLCM 특징을 추출합니다.
    
    Args:
        image: 그레이스케일 이미지 패치
        distances: GLCM 계산을 위한 거리 값들
        angles: GLCM 계산을 위한 각도 값들
    
    Returns:
        dict: GLCM 특징들 (contrast, dissimilarity, homogeneity, energy, correlation)
    """
    # 이미지가 너무 작으면 리사이즈
    if image.shape[0] < 8 or image.shape[1] < 8:
        image = cv2.resize(image, (8, 8))
    
    # GLCM 계산
    glcm = graycomatrix(image, distances, angles, 256, symmetric=True, normed=True)
    
    # GLCM 특징 추출
    features = {
        'contrast': float(graycoprops(glcm, 'contrast').mean()),
        'dissimilarity': float(graycoprops(glcm, 'dissimilarity').mean()),
        'homogeneity': float(graycoprops(glcm, 'homogeneity').mean()),
        'energy': float(graycoprops(glcm, 'energy').mean()),
        'correlation': float(graycoprops(glcm, 'correlation').mean())
    }
    
    return features

def get_object_patch(image: np.ndarray, x_center: float, y_center: float, width: float, height: float):
    """
    이미지에서 객체의 바운딩 박스 영역을 추출합니다.
    
    Args:
        image: 전체 이미지
        x_center, y_center: 객체의 중심 좌표 (normalized)
        width, height: 객체의 너비와 높이 (normalized)
    
    Returns:
        numpy.ndarray: 추출된 객체 이미지 패치
    """
    img_height, img_width = image.shape[:2]
    
    # 정규화된 좌표를 픽셀 좌표로 변환
    x_center_px = int(x_center * img_width)
    y_center_px = int(y_center * img_height)
    width_px = int(width * img_width)
    height_px = int(height * img_height)
    
    # 바운딩 박스 좌표 계산
    x1 = max(0, x_center_px - width_px // 2)
    y1 = max(0, y_center_px - height_px // 2)
    x2 = min(img_width, x_center_px + width_px // 2)
    y2 = min(img_height, y_center_px + height_px // 2)
    
    # 이미지 패치 추출
    patch = image[y1:y2, x1:x2]
    
    return patch

def yolo_to_json(image_path, label_path, output_path, distance_threshold=30):
    class_mapping = {
        0: {"class": "person", "state": "dynamic"},
        1: {"class": "rock", "state": "static"},
        2: {"class": "tree", "state": "static"},
        3: {"class": "stonewall", "state": "static"},
        4: {"class": "fence", "state": "static"},
        5: {"class": "pole", "state": "static"},
        6: {"class": "car", "state": "static"},
    }
    
    label_files = sorted(glob(os.path.join(label_path, "*.txt")))
    person_tracker = PersonTracker()
    
    for frame_idx, label_file in enumerate(label_files, 1):  # enumerate를 사용하여 프레임 번호 생성
        base_name = os.path.basename(label_file)
        image_name = os.path.splitext(base_name)[0] + ".png"
        image_file = os.path.join(image_path, image_name)
        
        # 이미지 로드
        try:
            image = cv2.imread(image_file)
            if image is None:
                raise FileNotFoundError(f"Cannot load image: {image_file}")
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        except Exception as e:
            print(f"Error loading image {image_file}: {str(e)}")
            continue
        
        nodes = []
        edges = []
        next_id = 1
        has_person = False
        
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()
                lines.sort(key=lambda x: float(x.strip().split()[0]) if x.strip() else 1)
                
                for line in lines:
                    if not line.strip():
                        continue
                    
                    try:
                        values = line.strip().split()
                        if len(values) != 5:
                            print(f"Warning: Skipping invalid line in {label_file}: {line}")
                            continue
                        
                        class_id, x_center, y_center, width, height = map(float, values)
                        class_id = int(class_id)
                        
                        if class_id not in class_mapping:
                            print(f"Warning: Skipping unknown class ID {class_id} in {label_file}")
                            continue
                        
                        # 좌표 변환
                        x = round(x_center * 100, 5)
                        y = round(y_center * 100, 5)
                        
                        object_patch = get_object_patch(gray_image, x_center, y_center, width, height)
                        glcm_features = extract_glcm_features(object_patch)
                        
                        if class_id == 0:
                            has_person = True
                            node_id = "0"
                            # person의 speed와 heading 계산
                            speed, heading = person_tracker.calculate_motion((x, y), node_id, frame_idx)
                        else:
                            node_id = str(next_id)
                            next_id += 1
                            # 정적 객체는 속도와 방향이 0
                            speed, heading = 0, 0
                        
                        node = {
                            "id": node_id,
                            "class": class_mapping[class_id]["class"],
                            "state": class_mapping[class_id]["state"],
                            "x": x,
                            "y": y,
                            "speed": speed,
                            "heading": heading,
                            "visual_features": glcm_features
                        }
                        nodes.append(node)
                        
                    except ValueError as e:
                        print(f"Warning: Error parsing line in {label_file}: {line}")
                        print(f"Error message: {str(e)}")
                        continue
            
            # missing person 처리
            if not has_person:
                nearest_obj = person_tracker.update(nodes, has_person, frame_idx)
                if nearest_obj:
                    # missing person의 speed와 heading도 계산
                    speed, heading = person_tracker.calculate_motion(
                        (nearest_obj["x"], nearest_obj["y"]), "0", frame_idx)
                    
                    missing_person = {
                        "id": "0",
                        "class": "person",
                        "state": "missing",
                        "x": nearest_obj["x"],
                        "y": nearest_obj["y"],
                        "speed": speed,
                        "heading": heading,
                        "visual_features": nearest_obj.get("visual_features", {})
                    }
                    nodes.insert(0, missing_person)
                    nearest_id = nearest_obj["id"]
            else:
                person_tracker.update(nodes, has_person, frame_idx)
                nearest_id = None
            
                    # FC 엣지 생성
            edge_id = 1
            for i in range(len(nodes)):
                for j in range(len(nodes)):
                    if i != j:
                        source_node = nodes[i]
                        target_node = nodes[j]
                        
                        # missing person 관련 엣지 처리
                        if not has_person and (source_node["id"] == "0" or target_node["id"] == "0"):
                            if (source_node["id"] == "0" and target_node["id"] == nearest_id):
                                # person -> nearest object: BACK 관계
                                spatial_relation = SpatialRelation.BACK
                            elif (target_node["id"] == "0" and source_node["id"] == nearest_id):
                                # nearest object -> person: FRONT 관계
                                spatial_relation = SpatialRelation.FRONT
                            else:
                                # 다른 객체들과는 NONE 관계
                                spatial_relation = SpatialRelation.NONE
                        else:
                            # 일반적인 경우의 공간적 관계 계산
                            rel_x = target_node["x"] - source_node["x"]
                            rel_y = target_node["y"] - source_node["y"]
                            distance = calculate_distance(
                                source_node["x"], source_node["y"],
                                target_node["x"], target_node["y"]
                            )
                            spatial_relation = determine_spatial_relation(
                                rel_x, rel_y, distance, distance_threshold)
                        
                        edges.append({
                            "id": f"e{edge_id}",
                            "source": source_node["id"],
                            "target": target_node["id"],
                            "position": spatial_relation.name.lower()
                        })
                        edge_id += 1
        
            # [JSON 저장 부분은 동일]
            
            json_data = {
                "Image": image_name,
                "label": base_name,
                "frame_no": str(frame_idx).zfill(4),  # 4자리 숫자로 포맷팅 (예: "0001", "0002", ...)
                "nodes": nodes,
                "edges": edges
            }
            
            output_file = os.path.join(output_path, f"{os.path.splitext(base_name)[0]}.json")
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(json_data, f, indent=2)
                
            print(f"Successfully processed {label_file}")
            
        except Exception as e:
            print(f"Error processing file {label_file}: {str(e)}")
            continue

if __name__ == "__main__":
    image_path = "./source/images"
    label_path = "./source/labels"
    output_path = "./output"
    distance_threshold = 30
    
    os.makedirs(output_path, exist_ok=True)
    yolo_to_json(image_path, label_path, output_path, distance_threshold)

### 1.3.1 JSON To GEXF
GEXF (Graph Exchange XML Format)는 그래프 데이터를 표현하기 위한 XML 기반 파일 형식으로, Python의 NetworkX 라이브러리를 사용하여 읽을 수 있습니다.
아래 코드를 사용하여 JSON 파일을 GEXF로 변환할 수 있습니다.

In [ ]:
# =======================
# Graph_Converter.py
# 이 코드는 JSON 파일을 입력으로 받아, GEXF 포맷으로 변환하는 작업을 수행함.
# =======================

import json
import networkx as nx
from glob import glob
import os

def json_to_graph(json_file):
    """
    JSON 파일을 NetworkX 그래프로 변환합니다.
    """
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 방향성 그래프 생성
    G = nx.DiGraph()
    
    # 그래프 메타데이터 추가
    G.graph['image'] = data['Image']
    G.graph['label'] = data['label']
    G.graph['frame_no'] = data['frame_no']
    
    # 노드 추가
    for node in data['nodes']:
        node_attrs = {
            'class_name': node['class'],
            'state': node['state'],
            'x': node['x'],
            'y': node['y'],
            'speed': node['speed'],
            'heading': node['heading']
        }
        # 시각적 특징이 있는 경우 추가
        if 'visual_features' in node:
            node_attrs['visual_features'] = node['visual_features']
        
        G.add_node(node['id'], **node_attrs)
    
    # 엣지 추가
    for edge in data['edges']:
        G.add_edge(
            edge['source'],
            edge['target'],
            id=edge['id'],
            position=edge['position']
        )
    
    return G

def save_graph(G, output_dir, format='gexf'):
    """
    그래프를 지정된 형식으로 저장합니다.
    
    Args:
        G (nx.DiGraph): 저장할 그래프
        output_dir (str): 저장할 디렉토리 경로
        format (str): 저장 형식 ('gexf', 'graphml', 'gml', 'pkl' 중 하나)
    """
    frame_no = G.graph.get('frame_no', '0000')
    base_name = f"graph_{frame_no}"
    
    if format == 'gexf':
        nx.write_gexf(G, os.path.join(output_dir, f"{base_name}.gexf"))
    elif format == 'graphml':
        nx.write_graphml(G, os.path.join(output_dir, f"{base_name}.graphml"))
    elif format == 'gml':
        nx.write_gml(G, os.path.join(output_dir, f"{base_name}.gml"))
    elif format == 'pkl':
        nx.write_gpickle(G, os.path.join(output_dir, f"{base_name}.pkl"))
    else:
        raise ValueError(f"Unsupported format: {format}")

def process_json_files(json_dir, graph_dir, format='gexf'):
    """
    JSON 파일들을 처리하여 그래프로 변환하고 저장합니다.
    
    Args:
        json_dir (str): JSON 파일들이 있는 디렉토리 경로
        graph_dir (str): 그래프를 저장할 디렉토리 경로
        format (str): 그래프 저장 형식
    """
    # 출력 디렉토리 생성
    os.makedirs(graph_dir, exist_ok=True)
    
    # JSON 파일 목록 가져오기
    json_files = sorted(glob(os.path.join(json_dir, "*.json")))
    
    for json_file in json_files:
        try:
            # JSON을 그래프로 변환
            G = json_to_graph(json_file)
            
            # 그래프 저장
            save_graph(G, graph_dir, format)
            
            print(f"Successfully processed and saved graph for {json_file}")
            
        except Exception as e:
            print(f"Error processing {json_file}: {str(e)}")
            continue

if __name__ == "__main__":
    # 디렉토리 설정
    json_dir = ".source/output"  # JSON 파일이 있는 디렉토리
    graph_dir = ".source/graphs"  # 그래프를 저장할 디렉토리
    
    # JSON 파일들을 처리하여 그래프로 변환하고 저장
    process_json_files(json_dir, graph_dir, format='gexf')
    
    print(f"\nAll graphs have been saved to {graph_dir}")

### 1.4 Graph Check
이 챕터에서는 그래프 시각화 도구를 사용하여 그래프의 에지와 노드가 잘 생성되었는지 확인할 수 있는 도구에 대해 다룹니다.
![예제 이미지4](./source/examples/ex6_.png)

In [ ]:
# =======================
# Graph_Visualization.py
# 이 코드는 JSON 파일을 입력으로 받아 그래프를 시각화하여 보여줌.
# =======================

import sys
import json
import os
from PySide6.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                             QHBoxLayout, QLabel, QPushButton, QFileDialog, 
                             QGraphicsView, QGraphicsScene, QListWidget)
from PySide6.QtCore import Qt, QRectF, QPointF
from PySide6.QtGui import QPen, QBrush, QColor, QPainter, QFont

class NodeItem(QGraphicsScene):
    def __init__(self, node_data, parent=None):
        super().__init__(parent)
        self.node_data = node_data
        self.setSceneRect(0, 0, 800, 600)
        self.node_items = {}
        self.edge_items = []    
        self.draw_node()
        self.draw_edges()
    
    def draw_node(self):
        # 노드 색상 매핑
        color_map = {
            "person": QColor(255, 100, 100),  # 빨간색
            "rock": QColor(100, 100, 255),    # 파란색
            "tree": QColor(100, 255, 100)     # 초록색
        }
        
        # 노드 그리기
        for node in self.node_data["nodes"]:
            x = node["x"] * 8  # 화면 크기에 맞게 스케일 조정
            y = node["y"] * 6
            
            # 노드 생성
            node_color = color_map.get(node["class"], QColor(200, 200, 200))
            ellipse = self.addEllipse(x-20, y-20, 40, 40, 
                                    QPen(Qt.black), 
                                    QBrush(node_color))
            
            # 노드 텍스트 추가
            text = self.addText(f"{node['class']}\nID: {node['id']}")
            text.setDefaultTextColor(Qt.black)
            text.setPos(x-20, y-35)
            
            self.node_items[node["id"]] = (ellipse, text)
    
    def draw_edges(self):
        # 엣지 그리기
        for edge in self.node_data["edges"]:
            source = self.get_node_center(edge["source"])
            target = self.get_node_center(edge["target"])
            
            if source and target:
                # 엣지 선 그리기
                line = self.addLine(source[0], source[1], 
                                  target[0], target[1], 
                                  QPen(Qt.gray, 1, Qt.DashLine))
                
                # 엣지 레이블 추가
                mid_x = (source[0] + target[0]) / 2
                mid_y = (source[1] + target[1]) / 2
                text = self.addText(edge["position"])
                text.setDefaultTextColor(Qt.darkGray)
                text.setPos(mid_x, mid_y)
                
                self.edge_items.append((line, text))
    
    def get_node_center(self, node_id):
        if node_id in self.node_items:
            ellipse = self.node_items[node_id][0]
            rect = ellipse.rect()
            return (rect.x() + rect.width()/2, rect.y() + rect.height()/2)
        return None

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Graph Visualization Tool")
        self.setGeometry(100, 100, 1200, 800)
        
        # 메인 위젯 설정
        main_widget = QWidget()
        self.setCentralWidget(main_widget)
        layout = QHBoxLayout(main_widget)
        
        # 왼쪽 패널
        left_panel = QWidget()
        left_layout = QVBoxLayout(left_panel)
        
        # 파일 선택 버튼
        self.file_btn = QPushButton("Open JSON File")
        self.file_btn.clicked.connect(self.open_file)
        left_layout.addWidget(self.file_btn)
        
        # 파일 목록
        self.file_list = QListWidget()
        self.file_list.itemClicked.connect(self.load_selected_file)
        left_layout.addWidget(self.file_list)
        
        # 정보 표시 레이블
        self.info_label = QLabel()
        self.info_label.setWordWrap(True)
        left_layout.addWidget(self.info_label)
        
        # 오른쪽 패널 (그래프 표시 영역)
        self.view = QGraphicsView()
        self.view.setRenderHint(QPainter.Antialiasing)
        self.view.setViewportUpdateMode(QGraphicsView.FullViewportUpdate)
        self.view.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        self.view.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        
        # 레이아웃 설정
        layout.addWidget(left_panel, 1)
        layout.addWidget(self.view, 4)
        
        self.current_dir = ""
    
    def open_file(self):
        file_name, _ = QFileDialog.getOpenFileName(
            self, "Open JSON File", "", "JSON Files (*.json)")
        
        if file_name:
            self.current_dir = os.path.dirname(file_name)
            self.update_file_list()
            self.load_json_file(file_name)
    
    def update_file_list(self):
        self.file_list.clear()
        if self.current_dir:
            json_files = [f for f in os.listdir(self.current_dir) 
                         if f.endswith('.json')]
            self.file_list.addItems(json_files)
    
    def load_selected_file(self, item):
        """리스트에서 선택된 파일을 로드합니다."""
        if self.current_dir and item:
            file_path = os.path.join(self.current_dir, item.text())
            self.load_json_file(file_path)
    
    def load_json_file(self, file_name):
        try:
            with open(file_name, 'r') as f:
                data = json.load(f)
                self.show_graph(data)
                
                # 정보 업데이트
                info_text = f"Image: {data['Image']}\n"
                info_text += f"Label: {data['label']}\n"
                info_text += f"Frame: {data['frame_no']}\n"
                info_text += f"Nodes: {len(data['nodes'])}\n"
                info_text += f"Edges: {len(data['edges'])}"
                self.info_label.setText(info_text)
        except Exception as e:
            self.info_label.setText(f"Error loading file: {str(e)}")
    
    def show_graph(self, data):
        """JSON 데이터로 그래프를 그립니다."""
        try:
            scene = NodeItem(data)
            self.view.setScene(scene)
            # 뷰 크기에 맞게 조정
            self.view.fitInView(scene.sceneRect(), Qt.KeepAspectRatio)
        except Exception as e:
            self.info_label.setText(f"Error displaying graph: {str(e)}")
    
    def resizeEvent(self, event):
        """창 크기가 변경될 때 그래프 크기도 조정합니다."""
        super().resizeEvent(event)
        if self.view.scene():
            self.view.fitInView(self.view.scene().sceneRect(), Qt.KeepAspectRatio)

def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

## 2. Model Training
이 챕터에서는 모델 구조 생성 및 학습, 시각화 및 평가의 전 과정을 하나의 코드로 다루고 있습니다.

In [ ]:

# =======================
# model.py
# 이 코드는 모델 구조 생성 및 학습, 시각화 및 평가까지의 작업을 수행함.
# =======================

import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
from glob import glob

# Temporal Graph Network with GAT (Graph Attention Network)
class TemporalGraphNetworkGAT(nn.Module):
    def __init__(self, feature_dim, hidden_dim, edge_dim, num_frames_predict=5, heads=4):
        super().__init__()
        self.num_frames_predict = num_frames_predict  # 예측할 프레임 수
        
        # GAT 레이어 정의
        self.gat1 = GATConv(feature_dim, hidden_dim, heads=heads, concat=False)  # 첫 번째 GAT 레이어
        self.gat2 = GATConv(hidden_dim, hidden_dim, heads=heads, concat=False)  # 두 번째 GAT 레이어
        
        # 엣지 특징을 임베딩하기 위한 선형 레이어
        self.edge_embedding = nn.Linear(edge_dim, hidden_dim)
        
        # 시간적 정보를 처리하기 위한 GRU (Gated Recurrent Unit)
        self.gru = nn.GRU(hidden_dim, hidden_dim, num_layers=2, batch_first=True)
        
        # 예측기: 최종 출력을 생성하는 MLP (Multi-Layer Perceptron)
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2 * num_frames_predict)  # 각 노드에 대해 2D 좌표를 예측
        )

    def forward(self, x, edge_index_seq, edge_feat_seq):
        batch_size = x.size(0)  # 배치 크기
        num_nodes = x.size(1)  # 노드 수
        time_steps = x.size(2)  # 시간 스텝 수
        
        spatial_embeddings = []  # 각 시간 스텝의 공간적 임베딩을 저장할 리스트

        # 각 시간 스텝에 대해 처리
        for t in range(time_steps):
            batch_embeddings = []  # 각 배치의 임베딩을 저장할 리스트
            for b in range(batch_size):
                x_t = x[b, :, t]  # 현재 배치와 시간 스텝의 노드 특징
                edge_index = edge_index_seq[b][t].to(x.device)  # 엣지 인덱스
                edge_feat = edge_feat_seq[b][t].to(x.device)  # 엣지 특징
                
                # 엣지 특징 임베딩
                edge_feat_embedded = self.edge_embedding(edge_feat)
                # 첫 번째 GAT 레이어 적용
                h = self.gat1(x_t, edge_index)
                h = torch.relu(h)  # 활성화 함수 적용
                # 두 번째 GAT 레이어 적용
                h = self.gat2(h, edge_index)
                batch_embeddings.append(h)
            
            # 현재 시간 스텝의 배치 임베딩을 스택으로 변환
            timestep_embedding = torch.stack(batch_embeddings)
            spatial_embeddings.append(timestep_embedding)
        
        # 공간적 임베딩을 시간적 차원으로 스택
        spatial_temporal = torch.stack(spatial_embeddings, dim=1)
        spatial_temporal = spatial_temporal.transpose(1, 2)  # 차원 재배열
        spatial_temporal = spatial_temporal.reshape(batch_size * num_nodes, time_steps, -1)  # GRU 입력 형태로 변환
        
        # GRU를 통해 시간적 정보 처리
        output, _ = self.gru(spatial_temporal)
        final_hidden = output[:, -1, :]  # 마지막 시간 스텝의 은닉 상태
        
        # 예측기로 최종 출력 생성
        predictions = self.predictor(final_hidden)
        predictions = predictions.view(batch_size, num_nodes, self.num_frames_predict, 2)  # 출력 형태 변환
        
        return predictions

# 모든 trajectory 데이터를 로드하는 함수
def load_all_trajectories(base_dir, trajectory_ids=[1, 2, 3, 4, 7, 8], sequence_length=10):
    """
    여러 trajectory 폴더의 그래프 시퀀스를 로드합니다.
    
    Args:
        base_dir: trajectory 폴더들이 있는 기본 디렉토리
        trajectory_ids: 처리할 trajectory 번호 리스트
        sequence_length: 입력으로 사용할 연속된 프레임 수
    """
    all_sequences = []
    
    for traj_id in trajectory_ids:
        print(f"Loading trajectory_{traj_id}...")
        graph_dir = os.path.join(base_dir, f"trajectory_{traj_id}", "graphs")
        
        # 해당 디렉토리가 존재하는지 확인
        if not os.path.exists(graph_dir):
            print(f"Warning: {graph_dir} does not exist. Skipping...")
            continue
            
        # 그래프 파일 목록 가져오기
        graph_files = sorted(glob(os.path.join(graph_dir, "*.gexf")))
        
        if not graph_files:
            print(f"Warning: No graph files found in {graph_dir}. Skipping...")
            continue
            
        print(f"Found {len(graph_files)} graph files in trajectory_{traj_id}")
        
        # 시퀀스 생성
        for i in range(len(graph_files) - sequence_length - 5):
            sequence = []
            future_sequence = []
            
            # 입력 시퀀스 로드
            for j in range(sequence_length):
                G = nx.read_gexf(graph_files[i + j])
                sequence.append(G)
                
            # 미래 프레임 로드
            for j in range(5):
                G = nx.read_gexf(graph_files[i + sequence_length + j])
                future_sequence.append(G)
                
            all_sequences.append({
                'trajectory_id': traj_id,
                'input_sequence': sequence,
                'future_sequence': future_sequence
            })
            
        print(f"Created {len(all_sequences)} sequences from trajectory_{traj_id}")
    
    print(f"\nTotal sequences created: {len(all_sequences)}")
    return all_sequences

# 시퀀스를 패딩하는 함수
def pad_sequence(features, max_nodes):
    """
    시퀀스를 지정된 최대 노드 수에 맞게 패딩합니다.
    """
    current_nodes = features.size(0)
    if current_nodes < max_nodes:
        # [node_num, timesteps, feature_dim] -> [max_nodes, timesteps, feature_dim]
        padding = torch.zeros(max_nodes - current_nodes, *features.size()[1:])
        return torch.cat([features, padding], dim=0)
    return features

# 그래프 시퀀스에서 특징을 추출하는 함수
def prepare_features(graph_sequence, max_nodes=15):
    sequence = graph_sequence['input_sequence']
    future = graph_sequence['future_sequence']
    
    features = []
    edge_indices = []
    edge_features = []
    
    # 첫 번째 그래프의 노드 수를 사용
    num_nodes = len(sequence[0].nodes())
    
    for G in sequence:
        node_feats = []
        for node in sorted(G.nodes()):
            node_data = G.nodes[node]
            try:
                if isinstance(node_data['visual_features'], str):
                    visual_features = eval(node_data['visual_features'])
                else:
                    visual_features = node_data['visual_features']
                visual_feat = np.array([
                    visual_features['contrast'],
                    visual_features['dissimilarity'],
                    visual_features['homogeneity'],
                    visual_features['energy'],
                    visual_features['correlation']
                ])
            except Exception as e:
                print(f"Error parsing visual features for node {node}: {str(e)}")
                visual_feat = np.zeros(5)
            
            pos = np.array([float(node_data['x'])/100, float(node_data['y'])/100])
            node_feats.append(np.concatenate([visual_feat, pos]))
        
        # 노드 특징을 미리 numpy 배열로 변환
        node_feats = np.array(node_feats)
        
        # 패딩 적용
        if len(node_feats) < max_nodes:
            padding = np.zeros((max_nodes - len(node_feats), node_feats.shape[1]))
            node_feats = np.vstack([node_feats, padding])
        
        features.append(torch.FloatTensor(node_feats))
        
        # 엣지 처리
        edge_idx = []
        edge_feat = []
        for e in G.edges(data=True):
            edge_idx.append([int(e[0]), int(e[1])])
            if e[2]['position'] == 'none':
                edge_feat.append([0] * 8)
            else:
                position_feat = [1 if e[2]['position'] == pos else 0 
                               for pos in ['front', 'back', 'left', 'right', 'front_left', 
                                         'front_right', 'back_left', 'back_right']]
                edge_feat.append(position_feat)
        
        edge_indices.append(torch.LongTensor(edge_idx).t())
        edge_features.append(torch.FloatTensor(edge_feat))
    
    future_positions = []
    for G in future:
        positions = []
        for node in sorted(G.nodes()):
            node_data = G.nodes[node]
            pos = [float(node_data['x']), float(node_data['y'])]
            positions.append(pos)
        
        # 패딩 적용
        while len(positions) < max_nodes:
            positions.append([0, 0])
        
        future_positions.append(positions)
    
    return {
        'features': torch.stack(features).transpose(0, 1),  # [nodes, timesteps, features]
        'edge_index': edge_indices,
        'edge_features': edge_features,
        'future_positions': torch.FloatTensor(future_positions).transpose(0, 1),  # [nodes, timesteps, 2]
        'num_nodes': torch.tensor(num_nodes)  # 스칼라 값
    }

# 데이터로더를 위한 커스텀 collate 함수
def custom_collate_fn(batch):
    features = torch.stack([item['features'] for item in batch])
    edge_indices = [[tensor for tensor in item['edge_index']] for item in batch]
    edge_features = [[tensor for tensor in item['edge_features']] for item in batch]
    future_positions = torch.stack([item['future_positions'] for item in batch])  # [batch, nodes, timesteps, 1, 2]
    num_nodes = torch.stack([item['num_nodes'] for item in batch])
    
    return {
        'features': features,
        'edge_index': edge_indices,
        'edge_features': edge_features,
        'future_positions': future_positions,
        'num_nodes': num_nodes
    }

# 학습 에포크를 처리하는 함수
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        features = batch['features'].to(device)
        edge_indices = batch['edge_index']
        edge_features = batch['edge_features']
        future_positions = batch['future_positions'].to(device)
        num_nodes = batch['num_nodes'].to(device)  # [batch_size]
        
        optimizer.zero_grad()
        predictions = model(features, edge_indices, edge_features)
        
        loss = 0
        for i in range(predictions.size(0)):  # batch 크기만큼 반복
            n = int(num_nodes[i])  # 각 배치의 실제 노드 수
            pred = predictions[i, :n]  # [n, num_frames_predict, 2]
            target = future_positions[i, :n]  # [n, num_frames_predict, 2]
            loss += criterion(pred, target)
        loss = loss / predictions.size(0)  # 배치 크기로 나누기
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 검증 함수
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            features = batch['features'].to(device)
            edge_indices = batch['edge_index']
            edge_features = batch['edge_features']
            future_positions = batch['future_positions'].to(device)
            num_nodes = batch['num_nodes'].to(device)
            
            predictions = model(features, edge_indices, edge_features)
            
            loss = 0
            for i in range(predictions.size(0)):
                n = int(num_nodes[i])
                pred = predictions[i, :n]
                target = future_positions[i, :n]
                loss += criterion(pred, target)
            loss = loss / predictions.size(0)
            
            total_loss += loss.item()
    
    return total_loss / len(dataloader)

# 테스트 함수
def test_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            features = batch['features'].to(device)
            edge_indices = batch['edge_index']
            edge_features = batch['edge_features']
            future_positions = batch['future_positions'].to(device)
            num_nodes = batch['num_nodes'].to(device)
            
            predictions = model(features, edge_indices, edge_features)
            
            # 패딩된 노드 제외하고 실제 노드만 저장
            batch_predictions = []
            batch_targets = []
            for i in range(len(num_nodes)):
                n = int(num_nodes[i].item())
                pred = predictions[i, :n, :, :]
                target = future_positions[i, :n, :, :]
                batch_predictions.append(pred)
                batch_targets.append(target)
                loss = criterion(pred, target)
                total_loss += loss.item()
            
            all_predictions.extend(batch_predictions)
            all_targets.extend(batch_targets)
    
    return (total_loss / len(dataloader), 
            torch.cat(all_predictions, dim=0), 
            torch.cat(all_targets, dim=0))

# 예측 결과 시각화 함수
def visualize_trajectories(predictions, targets, num_samples=5):
    for i in range(min(num_samples, predictions.shape[0])):
        plt.figure(figsize=(8, 8))
        plt.scatter(targets[i, :, 0], targets[i, :, 1], 
                   label="Ground Truth", color="blue", alpha=0.6)
        plt.scatter(predictions[i, :, 0], predictions[i, :, 1], 
                   label="Predictions", color="red", alpha=0.6)
        plt.title(f"Trajectory Prediction Sample {i+1}")
        plt.xlabel("X Position")
        plt.ylabel("Y Position")
        plt.legend()
        plt.grid(True)
        plt.savefig()
        # plt.show()

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # 전체 trajectory 데이터 로드
    print("Loading all trajectory sequences...")
    base_dir = "."  # 현재 디렉토리 기준
    sequences = load_all_trajectories(base_dir)
    
    # 데이터가 없는 경우 처리
    if not sequences:
        print("No valid sequences found. Exiting...")
        return
        
    # 데이터 섞기 (다른 trajectory의 데이터가 고르게 분포하도록)
    np.random.shuffle(sequences)
    
    # 데이터셋 분할
    num_sequences = len(sequences)
    train_size = int(0.7 * num_sequences)
    val_size = int(0.15 * num_sequences)
    
    train_sequences = sequences[:train_size]
    val_sequences = sequences[train_size:train_size+val_size]
    test_sequences = sequences[train_size+val_size:]
    
    print(f"\nDataset split:")
    print(f"Train sequences: {len(train_sequences)}")
    print(f"Validation sequences: {len(val_sequences)}")
    print(f"Test sequences: {len(test_sequences)}")
    
    train_data = [prepare_features(seq) for seq in train_sequences]
    val_data = [prepare_features(seq) for seq in val_sequences]
    test_data = [prepare_features(seq) for seq in test_sequences]
    
    # 배치 사이즈
    batch_size = 8
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn
    )
    val_loader = torch.utils.data.DataLoader(
        val_data, batch_size=batch_size, collate_fn=custom_collate_fn
    )
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size, collate_fn=custom_collate_fn
    )
    
    feature_dim = train_data[0]['features'].shape[-1]

    # 모델 하이퍼파라미터
    model = TemporalGraphNetworkGAT(
        feature_dim=feature_dim,
        hidden_dim=128,
        edge_dim=8,
        num_frames_predict=5,
        heads=4
    ).to(device)
    
    criterion = nn.HuberLoss(delta=1.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    print("Starting training...")
    num_epochs = 50
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss = validate(model, val_loader, criterion, device)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}:")
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Validation Loss: {val_loss:.4f}")
    
    # 학습 곡선 시각화
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig()
    # plt.show()
    
    # 최적 모델로 테스트
    print("\nTesting best model...")
    model.load_state_dict(torch.load('best_model.pth'))
    test_loss, predictions, targets = test_model(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}")
    
    # 예측 결과 시각화
    print("\nVisualizing predictions...")
    visualize_trajectories(predictions, targets)

if __name__ == "__main__":
    main()

## 3. Out
- source 폴더에서는 원본 영상 데이터를 모델 학습에 필요한 형태로 바꾸는 과정을 수행할 수 있는 파일들이 들어 있습니다.

- model 폴더에는 논문 작성 시 실험했던 모델 코드 및 파일들이 들어 있습니다.

- codes 폴더에는 이 문서의 모든 원본 코드들이 들어 있습니다.

화이팅!